In [143]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from scipy.stats import skew
import os
import csv
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

url = "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"




columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin']



 df = pd.read_csv(url, delim_whitespace=True, names=columns, na_values='?') 



 
# 결측값 처리 (horsepower 컬럼에 결측값 존재)
df = df.dropna()

해당 소스를 활용하여 연비(Mile per gallen)를 예측 

mse를 구하라



1. ML (RF, DT, LR)

2. FCNN

3. 순환데이터로 변환후 CNN



각 단계별 검사후 다음단계 진행

In [97]:
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"

columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin', 'car name']

df = pd.read_csv(url, delim_whitespace=True, names=columns, na_values='?')

In [98]:
df

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790.0,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52.0,2130.0,24.6,82,2,vw pickup
395,32.0,4,135.0,84.0,2295.0,11.6,82,1,dodge rampage
396,28.0,4,120.0,79.0,2625.0,18.6,82,1,ford ranger


In [99]:
# 각 열의 결측치 수 확인
print(df.isnull().sum())

mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64


In [100]:
# 결측치가 있는 모든 행 출력
df[df.isnull().any(axis=1)]

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
32,25.0,4,98.0,NaN,2046.0,19.0,71,1,ford pinto
126,21.0,6,200.0,NaN,2875.0,17.0,74,1,ford maverick
330,40.9,4,85.0,NaN,1835.0,17.3,80,2,renault lecar deluxe
336,23.6,4,140.0,NaN,2905.0,14.3,80,1,ford mustang cobra
354,34.5,4,100.0,NaN,2320.0,15.8,81,2,renault 18i
374,23.0,4,151.0,NaN,3035.0,20.5,82,1,amc concord dl


In [101]:
df = df.dropna()
df

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790.0,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52.0,2130.0,24.6,82,2,vw pickup
395,32.0,4,135.0,84.0,2295.0,11.6,82,1,dodge rampage
396,28.0,4,120.0,79.0,2625.0,18.6,82,1,ford ranger


In [102]:
df.columns

Index(['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'model year', 'origin', 'car name'],
      dtype='object')

In [103]:
df['mpg'].value_counts()

13.0    20
14.0    19
18.0    17
15.0    16
26.0    14
        ..
31.9     1
16.9     1
18.2     1
22.3     1
44.0     1
Name: mpg, Length: 127, dtype: int64

In [104]:
df['car name'].value_counts()

amc matador            5
ford pinto             5
toyota corolla         5
toyota corona          4
amc hornet             4
                      ..
buick skyhawk          1
chevrolet monza 2+2    1
ford mustang ii        1
pontiac astro          1
chevy s-10             1
Name: car name, Length: 301, dtype: int64

In [105]:
df['origin'].value_counts()

1    245
3     79
2     68
Name: origin, dtype: int64

In [144]:
y = df['mpg']

X = df.drop(['mpg', 'car name'], axis = 1)

y, X

(0      18.0
 1      15.0
 2      18.0
 3      16.0
 4      17.0
        ... 
 393    27.0
 394    44.0
 395    32.0
 396    28.0
 397    31.0
 Name: mpg, Length: 392, dtype: float64,
      cylinders  displacement  horsepower  weight  acceleration  model year  \
 0            8         307.0       130.0  3504.0          12.0          70   
 1            8         350.0       165.0  3693.0          11.5          70   
 2            8         318.0       150.0  3436.0          11.0          70   
 3            8         304.0       150.0  3433.0          12.0          70   
 4            8         302.0       140.0  3449.0          10.5          70   
 ..         ...           ...         ...     ...           ...         ...   
 393          4         140.0        86.0  2790.0          15.6          82   
 394          4          97.0        52.0  2130.0          24.6          82   
 395          4         135.0        84.0  2295.0          11.6          82   
 396          4         12

In [145]:
from sklearn.model_selection import train_test_split

X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [149]:
from sklearn.linear_model import LogisticRegression

clf_lr = LogisticRegression(random_state = 0)
clf_lr.fit(X_train1, y_train1)

pred_lr = clf_lr.predict(X_test1)

print(accuracy_score(y_test1, pred_lr))
print(confusion_matrix(y_test1, pred_lr))

ValueError: Unknown label type: 'continuous'

In [110]:
y2 = pd.get_dummies(y1).values
y2.shape

(392, 127)

In [111]:
y2

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [121]:
y = df['mpg']

X = df.drop(['mpg', 'car name'], axis = 1)

X

,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,8,307.0,130.0,3504.0,12.0,70,1
1,8,350.0,165.0,3693.0,11.5,70,1
2,8,318.0,150.0,3436.0,11.0,70,1
3,8,304.0,150.0,3433.0,12.0,70,1
4,8,302.0,140.0,3449.0,10.5,70,1
...,...,...,...,...,...,...,...
393,4,140.0,86.0,2790.0,15.6,82,1
394,4,97.0,52.0,2130.0,24.6,82,2
395,4,135.0,84.0,2295.0,11.6,82,1
396,4,120.0,79.0,2625.0,18.6,82,1


In [123]:
y = pd.get_dummies(y).values
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [124]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [125]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD,Adam


model = Sequential()

model.add(Dense(8,input_shape=(7,),activation='tanh'))
model.add(Dense(30,activation='tanh'))
model.add(Dense(30,activation='tanh'))
model.add(Dense(1))

model.compile(optimizer = 'rmsprop', loss = 'mse', metrics = ['mse'])

model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_34 (Dense)            (None, 8)                 64        
                                                                 
 dense_35 (Dense)            (None, 30)                270       
                                                                 
 dense_36 (Dense)            (None, 30)                930       
                                                                 
 dense_37 (Dense)            (None, 1)                 31        
                                                                 
Total params: 1,295
Trainable params: 1,295
Non-trainable params: 0
_________________________________________________________________


In [126]:
history=model.fit(x=X_train2, y=y_train2, epochs=30, batch_size=32,validation_data= (X_test2,y_test2))
y_pred = model.predict(X_test2)

Epoch 1/30
10/10 [==============================] - 2s 50ms/step - loss: 0.9591 - mse: 0.9591 - val_loss: 0.0674 - val_mse: 0.0674
Epoch 2/30
10/10 [==============================] - 0s 15ms/step - loss: 0.0242 - mse: 0.0242 - val_loss: 0.0079 - val_mse: 0.0079
Epoch 3/30
10/10 [==============================] - 0s 14ms/step - loss: 0.0078 - mse: 0.0078 - val_loss: 0.0078 - val_mse: 0.0078
Epoch 4/30
10/10 [==============================] - 0s 15ms/step - loss: 0.0078 - mse: 0.0078 - val_loss: 0.0078 - val_mse: 0.0078
Epoch 5/30
10/10 [==============================] - 0s 14ms/step - loss: 0.0078 - mse: 0.0078 - val_loss: 0.0078 - val_mse: 0.0078
Epoch 6/30
10/10 [==============================] - 0s 14ms/step - loss: 0.0078 - mse: 0.0078 - val_loss: 0.0078 - val_mse: 0.0078
Epoch 7/30
10/10 [==============================] - 0s 14ms/step - loss: 0.0078 - mse: 0.0078 - val_loss: 0.0078 - val_mse: 0.0078
Epoch 8/30
10/10 [==============================] - 0s 14ms/step - loss: 0.0078 - m

In [128]:
y_pred = model.predict(X_test2)
mse = np.mean((y_test2 - y_pred)**2)
print(f'Mean Squared Error: {mse}')

4/4 [==============================] - 0s 6ms/step
Mean Squared Error: 0.010798281989991665


In [129]:
def split_sequence(sequence, n_steps):
    X, y = [], []
    for i in range(len(sequence)):
        # 끝에 도달하면 반복 종료
        end_ix = i + n_steps
        if end_ix > len(sequence) - 1:
            break
        # 입력과 출력 분리
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [130]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

n_steps = 3
X_seq, y_seq = split_sequence(X_scaled, n_steps)

In [131]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

In [132]:
X_train3.shape, X_test3.shape, y_train3.shape, y_test3.shape

((311, 3, 7), (78, 3, 7), (311, 7), (78, 7))

In [133]:
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Dense, LSTM, Dropout, RNN

model2 = Sequential()

model2.add(layers.Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model2.add(layers.MaxPooling1D(pool_size=2))
model2.add(layers.Conv1D(filters=128, kernel_size=1, activation='relu'))
model2.add(layers.GlobalMaxPooling1D())

model2.add(layers.Dense(50, activation='relu'))
model2.add(layers.Dense(1))

model2.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [135]:
history=model2.fit(x=X_train3, y=y_train3, epochs=30, batch_size=32,validation_data= (X_test3,y_test3))
y_pred = model2.predict(X_test3)

Epoch 1/30
10/10 [==============================] - 4s 64ms/step - loss: 0.9993 - accuracy: 0.0000e+00 - val_loss: 1.0277 - val_accuracy: 0.0000e+00
Epoch 2/30
10/10 [==============================] - 0s 16ms/step - loss: 0.9687 - accuracy: 0.0000e+00 - val_loss: 1.0102 - val_accuracy: 0.0000e+00
Epoch 3/30
10/10 [==============================] - 0s 15ms/step - loss: 0.9591 - accuracy: 0.0000e+00 - val_loss: 1.0190 - val_accuracy: 0.0000e+00
Epoch 4/30
10/10 [==============================] - 0s 19ms/step - loss: 0.9521 - accuracy: 0.0000e+00 - val_loss: 1.0189 - val_accuracy: 0.0000e+00
Epoch 5/30
10/10 [==============================] - 0s 17ms/step - loss: 0.9482 - accuracy: 0.0000e+00 - val_loss: 1.0305 - val_accuracy: 0.0000e+00
Epoch 6/30
10/10 [==============================] - 0s 17ms/step - loss: 0.9454 - accuracy: 0.0000e+00 - val_loss: 1.0235 - val_accuracy: 0.0000e+00
Epoch 7/30
10/10 [==============================] - 0s 16ms/step - loss: 0.9418 - accuracy: 0.0000e+00 - v

In [136]:
y_pred = model2.predict(X_test)
mse = np.mean((y_test - y_pred)**2)
print(f'Mean Squared Error: {mse}')

3/3 [==============================] - 0s 10ms/step
Mean Squared Error: 1.0504554167835107
